# 전체적인 데이터 이해
# EDA 목차

- index
    1. 패키지 불러오기
    2. 데이터 불러오기
    3. 데이터 정보
        - 3-1. 데이터 소개
        - 3-2. 결측치 확인
        - 3-3. 데이터 기본 전처리
        - 3-3. 이상치 확인
    4. 불만 제기 시간 확인
        - 4-1. 고객들의 시간별 불만 제기
    5. Err Data 관계 해석
        - 5-1. Errtype
        - 5-2. Errcode
        - 5-3. Fwver
        - 5-4. Model
    6. Quality Data 수치 해석
        - 6-1. Quality 값
        - 6-2. Fwver
    7. Err Data와 Quality Data 관계 해석
    8. 결과 정리

## 1. 패키지 불러오기

In [1]:
import os
import random 
import datetime as dt
import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# dataframe 보여줄 범위 설정

pd.options.display.max_columns=1000
pd.options.display.max_rows=200
pd.options.display.float_format = '{:.5f}'.format

## 2. 데이터 불러오기

In [4]:
load_path = '/content/drive/MyDrive/Colab Notebooks/data/235687_시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/'
save_path = '/content/drive/MyDrive/Colab Notebooks/data/235687_시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/result/'

In [5]:
train_err = pd.read_csv(load_path + 'train_err_data.csv')
train_qual = pd.read_csv(load_path + 'train_quality_data.csv')
train_prob = pd.read_csv(load_path + 'train_problem_data.csv')

In [6]:
test_err = pd.read_csv(load_path + 'test_err_data.csv')
test_qual = pd.read_csv(load_path + 'test_quality_data.csv')

## 3.데이터 정보
### 3-1 데이터 소개
- Error Data는 사람들이 에러를 접한 시간을 기준으로 **어떤 Model과 Fwver를 사용했는지와 어떤 Errtype과 Errcode를 접했는지**에 대한 내용입니다.  
  
- 변수는 총 6개이며 1600만개의 관측치가 user_id를 기준으로 시계열 데이터로 나와있습니다.  
    - user_id : 사용자 고유 ID
    - time : 에러가 발생한 시간
    - model : 에러가 발생한 모델명
    - fwver : 에러가 발생한 펌웨어 버전
    - errtype : 발생한 에러 타입
    - errcode : 발생한 에러 코드

In [7]:
train_err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 757.8+ MB


In [8]:
test_err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16532648 entries, 0 to 16532647
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 756.8+ MB


- quality data의 경우 사용자가 시스템을 사용하던 중 문제가 발생하면 **측정 가능한 지표**들을 문제발생시점부터 **2시간 단위로 수집**한 내용입니다.
    - user_id : 사용자 고유 ID
    - time : 퀄리티가 수집되기 시작한 시간
    - fwver : 퀄리티 수집 시작 시점 기준의 펌웨어 버전
    - quality : 에러 퀄리티 수치 (0~12, 총 13개 컬럼)

In [9]:
train_qual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       788544 non-null  object 
 3   quality_0   684192 non-null  float64
 4   quality_1   828624 non-null  int64  
 5   quality_2   788511 non-null  float64
 6   quality_3   828624 non-null  int64  
 7   quality_4   828624 non-null  int64  
 8   quality_5   828604 non-null  object 
 9   quality_6   828624 non-null  int64  
 10  quality_7   828624 non-null  object 
 11  quality_8   828624 non-null  object 
 12  quality_9   828624 non-null  object 
 13  quality_10  828624 non-null  object 
 14  quality_11  828624 non-null  int64  
 15  quality_12  828624 non-null  int64  
dtypes: float64(2), int64(8), object(6)
memory usage: 101.2+ MB


In [10]:
test_qual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        747972 non-null  int64  
 1   user_id     747972 non-null  int64  
 2   fwver       725208 non-null  object 
 3   quality_0   641388 non-null  float64
 4   quality_1   747961 non-null  object 
 5   quality_2   726857 non-null  float64
 6   quality_3   747972 non-null  int64  
 7   quality_4   747972 non-null  int64  
 8   quality_5   747928 non-null  object 
 9   quality_6   747972 non-null  int64  
 10  quality_7   747972 non-null  object 
 11  quality_8   747972 non-null  object 
 12  quality_9   747972 non-null  object 
 13  quality_10  747972 non-null  object 
 14  quality_11  747972 non-null  int64  
 15  quality_12  747972 non-null  int64  
dtypes: float64(2), int64(7), object(7)
memory usage: 91.3+ MB


- problem data는 불만을 제기한 사람들에 대한 데이터입니다.
    - user_id : 불만을 제기한 유저 ID
    - time : 불만 점수 시간
    - <u>**한 사용자가 불만을 여러번 제기할 수 있음**<u/>



In [11]:
train_prob.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5429 entries, 0 to 5428
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  5429 non-null   int64
 1   time     5429 non-null   int64
dtypes: int64(2)
memory usage: 85.0 KB


### 3-2. 결측치 확인
1. train_err_data - missing value  
    - errcode 결측치 존재(1개)

In [12]:
train_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     1
dtype: int64

In [13]:
train_err[train_err['errcode'].isnull()]

,user_id,time,model_nm,fwver,errtype,errcode
3825744,13639,20201121191718,model_2,04.33.1261,5,NaN


- 기록된 user_id와 time, model_nm, fwver, errtype이 같다면 errcode도 같을거라는 예상으로 결측치를 채웁니다.

In [14]:
# tre = train_err.iloc[:4000000, :].copy() # tre라는 변수를 만들어 train_err data에서 400만개의 데이터만 뽑아 복사하고
# tre_missing = tre.groupby(['user_id','time','fwver','errtype'])['errcode'].unique().to_frame() # user_id, time, fwver, errtype으로 묶어 errcode가 unique한 값들로 tre_missing이라는 dataframe을 생성

In [15]:
tre_missing

NameError: ignored

In [ ]:
tre_missing['errcode']

In [ ]:
# tre_missing['errcode'] = tre_missing['errcode'].apply(lambda x : len(x)) # errcode 컬럼 값을 바꿈

# ltwo = len(tre_missing[tre_missing['errcode'] != 1]) # errcode 컬럼의 값이 1이 아닌 것들 (동일한 특성을 갖는 row안에 다른 errcode)
# lone = len(tre_missing[tre_missing['errcode'] == 1]) # errcode 컬럼의 값이 1인 것들 (동일한 특성을 갖는 row안에 같은 errcode)
# print("동일한 특성을 갖는 Row안에 다른 ErrorCode가 발생할 확률 : %.3f%%"%(ltwo/lone * 100))

In [ ]:
tre_missing

In [ ]:
tre_missing['errcode'].unique()

In [ ]:
ltwo

In [ ]:
lone

In [ ]:
train_err[(train_err['user_id'] == 13639)&(train_err['fwver']=='04.33.1261')&(train_err['errtype']==5)&(train_err['time']==20201121191718)]

user_id가 13639이고, fwver은 04.33.1261이며 errtype은 5, 에러발생시간은 2020년 11월 21일 19시 17분 18초인 데이터의 비어있는 errcode를 같은 조건의 행에 있는 errcode로 대체

In [ ]:
train_err['errcode'] = train_err['errcode'].fillna('40013')

In [ ]:
train_err.isna().sum()

2. train_quality_data missing value
    - fwver 결측치 존재
    - quality_0, quality_2, quality_5 결측치 존재


In [ ]:
train_qual.isnull().sum()

In [16]:
# NaN 값일 경우의 quality가 측정된 기간
train_qual[train_qual['fwver'].isnull()].groupby('user_id')['time'].unique().to_frame()['time'].str.len().mean()

55.45762711864407

In [17]:
# NaN 값이 아닐 경우 quality가 측정된 기간
train_qual[~train_qual['fwver'].isnull()].groupby('user_id')['time'].unique().to_frame()['time'].str.len().mean()

7.9863813229571985

In [18]:
fwms_idx = train_qual[train_qual['fwver'].isnull()].index
train_qual = train_qual.drop(fwms_idx)

- quality_0, quality_2, quality_5 결측치 제거

In [19]:
for i in train_qual.columns[3:]:
    train_qual[i] = train_qual[i].fillna(train_qual[i].mode(0)[0])

- NaN 값을 갖는 경우 quality가 지속적으로 오랜기간 측정됨
- 반대로, NaN값을 갖지 않는 경우 상대적으로 적은기간 측정됨
- 따라서, NaN값을 갖는 경우 이상치에 해당 -> 해당 결측치 제거

3. test_error_data-missing value

In [20]:
test_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     4
dtype: int64

In [21]:
test_err[test_err['errcode'].isnull()]

,user_id,time,model_nm,fwver,errtype,errcode
937967,30820,20201115044317,model_2,04.33.1261,5,NaN
4038892,33681,20201103110259,model_2,04.33.1185,5,NaN
9486881,38991,20201127213838,model_2,04.33.1261,5,NaN
10425473,39894,20201128144712,model_1,04.16.3553,5,NaN


위에서 train data를 처리할 때 처럼 동일한 특성을 갖을 row라면 같은 에러코드를 가질 확률이 높음

In [22]:
te = test_err.iloc[:4000000,:].copy()
te_missing = te.groupby(['user_id','time','fwver','errtype'])['errcode'].unique().to_frame()
te_missing['errcode'] = te_missing['errcode'].apply(lambda x : len(x))

ltwo = len(te_missing[te_missing['errcode']!=1])
lone = len(te_missing[te_missing['errcode']==1])
print("동일한 특성을 갖는 Row안에 다른 ErrorCode가 발생할 확률 : %.3f%%"%(ltwo/lone * 100))

동일한 특성을 갖는 Row안에 다른 ErrorCode가 발생할 확률 : 0.054%


- 따라서 동일한 에러 코드로 결측치 교체

In [23]:
test_err.iloc[937967,5] = '40053'

In [24]:
test_err.iloc[4038892,5] = '40053'

In [25]:
test_err.iloc[9486881,5] = '40053'

In [26]:
test_err.iloc[10425473,5] = '-1010'

- train_err의 user는 15000명인데 test_err의 user는 14999명이므로 임의의 user를 1명 추가해줍니다.(최빈값으로 대체)

In [27]:
missing_value = pd.Series([43262, train_err.time.mode()[0], train_err.model_nm.mode()[0],
                train_err.fwver.mode()[0], train_err.errtype.mode()[0],
                train_err.errcode.mode()[0]], index = test_err.columns)

In [28]:
test_err = test_err.append(missing_value, ignore_index = True)

4. test_quality_data-missing value
    - fwver 결측치 존재
    - quality_0, quality_1, quality_2, quality_5 결측치 존재

In [29]:
test_qual.isnull().sum()

time               0
user_id            0
fwver          22764
quality_0     106584
quality_1         11
quality_2      21115
quality_3          0
quality_4          0
quality_5         44
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [30]:
# NaN 값일 경우의 quality가 측정된 기간
test_qual[test_qual['fwver'].isnull()].groupby('user_id')['time'].unique().to_frame()['time'].str.len().mean()

36.627450980392155

In [31]:
# NaN 값이 아닐 경우의 quality가 측정된 기간
test_qual[~test_qual['fwver'].isnull()].groupby('user_id')['time'].unique().to_frame()['time'].str.len().mean()

7.348095875410634

In [32]:
fwms_idx = test_qual[test_qual['fwver'].isnull()].index
test_qual = test_qual.drop(fwms_idx)

- quality_0, quality_1, quality_2, quality_5 결측치 제거

In [33]:
for i in test_qual.columns[3:]:
    test_qual[i] = test_qual[i].fillna(test_qual[i].mode(0)[0])

- 결측치가 제대로 제거(또는 대체)되었는지 확인

In [34]:
print(train_err.isnull().sum().sum(), end=", ")
print(test_err.isnull().sum().sum(), end=", ")
print(train_qual.isnull().sum().sum(), end=", ")
print(test_qual.isnull().sum().sum())

1, 0, 0, 0


- NaN 값을 갖는 경우 quality가 지속적으로 오랜기간 측정됨
- 반대로, NaN값을 갖지 않는 경우 상대적으로 적은기간 측정됨
- 따라서, NaN값을 갖는 경우 이상치에 해당 -> 해당 결측치 제거

### 3-3. 데이터 기본 전처리
1. 변수 타입 통일
    - time 변수의 타입을 int에서 datetime으로 변경

In [35]:
# time 변수 타입 변경 이전 데이터 저장
# train_err.to_csv(save_path + "train_err_data.csv", index=False)
# train_qual.to_csv(save_path + "train_quality_data.csv", index=False)
# train_prob.to_csv(save_path + "train_problem_data.csv", index=False)

# test_err.to_csv(save_path + "test_err_data.csv", index=False)
# test_qual.to_csv(save_path + "test_quality_data.csv", index=False)

In [36]:
def make_datetime(x): # datetime 데이터로 변환
    x = str(x)
    year = int(x[:4])
    month = int(x[4:6])
    day = int(x[6:8])
    hour = int(x[8:10])
    minute = int(x[10:12])
    sec = int(x[12:])
    return dt.datetime(year, month, day, hour, minute, sec)

In [37]:
train_err.time = train_err.time.apply(lambda x : make_datetime(x))

In [38]:
train_qual = train_qual.sort_values(['user_id','time']).reset_index(drop=True)
train_qual.time = train_qual.time.apply(lambda x : make_datetime(x))

In [39]:
train_prob = train_prob.sort_values(['user_id','time']).reset_index(drop=True)
train_prob.time = train_prob.time.apply(lambda x : make_datetime(x))

In [40]:
test_err.time = test_err.time.apply(lambda x : make_datetime(x))

In [41]:
test_qual.time = test_qual.time.apply(lambda x : make_datetime(x))

In [42]:
train_qual

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,2020-11-29 09:00:00,10000,05.15.2138,0.00000,0,0.00000,0,0,0,0,0,0,0,4,0,0
1,2020-11-29 09:00:00,10000,05.15.2138,0.00000,0,0.00000,0,0,0,0,0,0,0,4,0,0
2,2020-11-29 09:00:00,10000,05.15.2138,0.00000,0,0.00000,0,0,0,0,0,0,0,4,0,0
3,2020-11-29 09:00:00,10000,05.15.2138,0.00000,0,0.00000,0,0,0,0,0,0,0,4,0,0
4,2020-11-29 09:00:00,10000,05.15.2138,0.00000,0,0.00000,0,0,0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788539,2020-11-24 03:30:00,24997,04.22.1778,0.00000,0,0.00000,0,0,1,0,0,0,0,17,0,0
788540,2020-11-24 03:30:00,24997,04.22.1778,0.00000,0,0.00000,0,0,0,0,0,0,0,17,0,0
788541,2020-11-24 03:30:00,24997,04.22.1778,0.00000,0,0.00000,0,0,3,0,0,0,0,17,0,0
788542,2020-11-24 03:30:00,24997,04.22.1778,0.00000,0,0.00000,0,0,0,0,0,0,0,17,0,0


- quality 변수의 타입을 object(또는 float)에서 int로 변경

In [43]:
def str2int(x):
    if type(x) == str:
        x = x.replace(",","")
        x = int(x)
        return x
    else:
        x = int(x)
        return x

In [44]:
for i in train_qual.columns[3:]:
    train_qual[i] = train_qual[i].apply(lambda x :str2int(x))

In [45]:
for i in test_qual.columns[3:]:
    test_qual[i] = test_qual[i].apply(lambda x : str2int(x))

## 3-4. 이상치 확인
1. fwver의 이상치 확인
    - 펌웨어 버전의 운용 기간이 **비정상적으로 짧은 것**들이 존재하였음  
        -> 해당 펌웨어 버전을 사용한 유전는 펌웨어 버전당 한 명뿐이었음  
        -> 따라서, 이러한 버전을 사용한 유저는 이상치에 속한다고 보고 데이터를 확인하였음

In [46]:
temp = train_err[['time','fwver']].groupby('fwver').time.unique() # train_err_data에 time과 fwver 컬럼을 가져와 fwver를 기준으로 묶고 time이 unique한 것들을 temp라는 변수에 담음
df = pd.DataFrame(columns = ['fwver','start','end']) # 새로운 dataframe 생성
for i in range(len(temp)):
    temp.values[i].sort() # temp에 담긴 값들을 정렬
    df = df.append(pd.Series([temp.index[i], temp.values[i][0], temp.values[i][-1]], index=df.columns),
                   ignore_index = True) # 펌웨어 운용의 시작날짜와 끝난날짜를 df에 담음

In [47]:
def find_abnormal(x): # x를 살펴본 기간 차이
    for fwver, st, ed in tqdm(df.values):
        if((ed - st).days < x):
            user = train_err[train_err['fwver']==fwver].user_id.unique()

In [48]:
find_abnormal(3) # fwver : 펌웨어 버전 # per_num : 사용한 유저의 수 # id : 사용한 유저의 id

100%|██████████| 37/37 [00:03<00:00, 11.61it/s]


- 18142번 유저는 약 3분동안 혼자 05.15.2090 버전을 사용하였음 → 퀄리티값에 이상이 없음, 에러 로그 존재함 → 불만 제기 안함
- 19831번 유저는 하루~이틀동안 혼자 04.22.1656 버전을 사용하였음 → 퀄리티값에 이상이 있음, 에러 로그 존재함 → 불만 제기
- 24279번 유저는 약 18분동안 혼자 05.15.2092 버전을 사용하였음 → 퀄리티값에 이상이 없음, 에러 로그 존재함 → 불만 제기



In [49]:
abnormal_lst = [18142, 19831, 24279]
display(train_prob[train_prob['user_id'].isin(abnormal_lst)]) # 해당 유저들이 불만을 제기했는지 확인
# display(train_qual[train_qual['user_id'].isin(abnormal_lst)]) # 해당 유저들의 quality 값에 이상이 있는지 확인
# display(train_err[train_err['user_id'].isin(abnormal_lst)][:50]) # 해당 유저의 error data 확인

,user_id,time
3499,19831,2020-11-23 11:00:00
5180,24279,2020-11-13 11:00:00


24279번 유저는 이상치라고 생각할 수 있다.  
(추후 분석을 통해 이상치임을 더 확실하게 알 수 있다. 따라서 추후 분석 이전에는 제거하지 않는다.)

2. error log 이상치 확인
    - 에러 로그양이 적음에도 불만을 제기한 사람들이 존재

In [50]:
log_lst = list(train_err.groupby('user_id')['errtype'].count().to_frame().sort_values(by='errtype').reset_index().user_id.unique())[:100] # 에러 로그양이 가장 적은 유저 100명

print(len(train_prob[train_prob['user_id'].isin(log_lst)])) # 에러 로그가 적은 유저 100명 중, 불만을 제기한 유저
print(list(train_prob[train_prob['user_id'].isin(log_lst)].user_id.unique())) # 에러 로그가 적은 유저 100명의 리스트

5
[12623, 16980, 20271, 20300, 21040]


- 위 5명의 error data & quality data를 살펴봄

In [51]:
display(train_err[train_err['user_id'] == 16980])
display(train_qual[train_qual['user_id'] == 16980])
print("\n")
display(train_err[train_err['user_id'] == 20271])
display(train_qual[train_qual['user_id'] == 20271])
print("\n")
display(train_err[train_err['user_id'] == 12623])
display(train_qual[train_qual['user_id'] == 12623])
print("\n")
display(train_err[train_err['user_id'] == 21040])
display(train_qual[train_qual['user_id'] == 21040])
print("\n")
display(train_err[train_err['user_id'] == 20300])
display(train_qual[train_qual['user_id'] == 20300])
print("\n")

,user_id,time,model_nm,fwver,errtype,errcode
7787926,16980,2020-11-17 22:44:56,model_6,10,5,S-61001
7787927,16980,2020-11-17 22:44:59,model_6,10,5,S-61001


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


,user_id,time,model_nm,fwver,errtype,errcode
11112255,20271,2020-11-04 22:11:48,model_6,10,5,S-61001
11112256,20271,2020-11-05 21:53:00,model_6,10,5,S-61001


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


,user_id,time,model_nm,fwver,errtype,errcode
2750266,12623,2020-11-04 21:12:52,model_6,10,5,S-61001
2750267,12623,2020-11-04 21:13:52,model_6,10,5,S-61001


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


,user_id,time,model_nm,fwver,errtype,errcode
12210417,21040,2020-11-04 22:43:14,model_6,10,5,B-A8002
12210418,21040,2020-11-04 22:43:47,model_6,10,5,B-A8002


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


,user_id,time,model_nm,fwver,errtype,errcode
11144486,20300,2020-11-07 20:04:39,model_3,05.15.2138,26,1
11144487,20300,2020-11-07 20:04:46,model_3,05.15.2138,12,1
11144488,20300,2020-11-07 20:04:46,model_3,05.15.2138,11,1
11144489,20300,2020-11-07 21:38:48,model_3,05.15.2138,4,0


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


에러 로그가 적은 유저들은 **퀄리티가 측정되지 않았다**는 공통점이 발견되었다.
(추후 분석을 통해 이상치임을 더 확실하게 알 수 있다. 따라서 추후 분석 이전에는 제거하지 않는다.)  
(그리고 이후 error data와 quality data간의 관계 분석에 사용할 수 있다.)

- 전처리된 데이터 저장

In [52]:
# time 변수 타입 변경 이전 + 그 외에는 전처리된(결측치, 이상치 처리) 데이터 저장
# train_err.to_csv(save_path + "train_err_data_time.csv", index=False)
# train_qual.to_csv(save_path + "train_quality_data_time.csv", index=False)
# train_prob.to_csv(save_path + "train_problem_data_time.csv", index=False)

# test_err.to_csv(save_path + "test_err_data_time.csv", index=False)
# test_qual.to_csv(save_path + "test_quality_data_time.csv", index=False)

# 4. 불만 제기 시간 확인

---
## 4-1. 고객들이 언제, 무슨요일, 어느시간에 불만을 제기했는지 확인

In [53]:
# Datetime으로 Type이 변경되기 전 데이터 로드
train_err = pd.read_csv(save_path+"train_err_data.csv")
train_qual = pd.read_csv(save_path+"train_quality_data.csv")
train_prob = pd.read_csv(save_path+"train_problem_data.csv")

test_err = pd.read_csv(save_path+"test_err_data.csv")
test_qual = pd.read_csv(save_path+"test_quality_data.csv")

In [54]:
# 불만 제기 고객 확인
problem_user = train_prob.user_id.unique()
no_problem_ser = list(set(train_err.user_id.unique()) - set(problem_user))

In [55]:
def make_weekday(col): # 0: 월요일, 1: 화요일, 2: 수요일, 3: 목요일, 4: 금요일, 5: 토요일, 6: 일요일
    col = str(col)
    year = int(col[:4])
    month = int(col[4:6])
    day = int(col[6:8])
    weekday = dt.datetime(year,month,day).weekday()

    return weekday

In [56]:
def make_hour(col): # 시간대별
    col = str(col)
    hour = int(col[8:10])
# hour = str(hour) + "시"
    return hour

In [57]:
train_prob['weekday'] = train_prob.time.apply(lambda x : make_weekday(x))
train_prob['hour'] = train_prob.time.apply(lambda x : make_hour(x))

In [58]:
train_prob['weekday'].value_counts() # 요일별 불만제기

0    1332
2     868
4     853
1     809
3     809
5     431
6     327
Name: weekday, dtype: int64

월 -> 수 -> 금 -> 화 -> 목 -> 토 -> 일 순

In [59]:
train_prob['hour'].value_counts() # 시간별 불만 제기

12    627
11    600
13    518
19    507
18    492
15    489
16    460
17    432
22    233
20    233
14    228
21    204
23    175
0     108
1      67
2      15
10     14
9       8
5       5
8       5
3       4
4       3
7       1
6       1
Name: hour, dtype: int64

점심 -> 저녁 -> 오후 -> 밤 -> 새벽 -> 아침 순

In [60]:
weekend = train_prob[(train_prob['weekday'] == 5) | (train_prob['weekday'] == 6)]
weekdays = train_prob[(train_prob['weekday'] != 5) & (train_prob['weekday'] != 6)]

In [61]:
week_ratio = len(weekend) / len(train_prob) #주말 제기 확률
print("평일에 불만을 제기할 확률 : {0}%\n주말에 불만을 제기할 확률 : {1}%".
      format(round((1-week_ratio)*100, 3), (week_ratio)*100, 3))

평일에 불만을 제기할 확률 : 86.038%
주말에 불만을 제기할 확률 : 13.962055627187327%


- 주말보다는 평일에 불만 제기가 많음

In [62]:
weekend_open = weekend[((weekend['weekday'] != 6) >= 13)]
weekdays_open = weekdays[((weekdays['hour'] >= 9) <= 18)]
print("운영시간 중 불만 제기 확률 : {0}%".format(round((len(weekdays_open)+ len(weekend_open)) / len(train_prob) * 100, 3)))

운영시간 중 불만 제기 확률 : 86.038%


In [63]:
# 다시 Datetime으로 Type이 변경된 데이터 로드
train_err = pd.read_csv(save_path+"train_err_data_time.csv")
train_qual = pd.read_csv(save_path+"train_quality_data_time.csv")
train_prob = pd.read_csv(save_path+"train_problem_data_time.csv")

test_err = pd.read_csv(save_path+"test_err_data_time.csv")
test_qual = pd.read_csv(save_path+"test_quality_data_time.csv")

# 5. Error Data 해석
에러가 많이 발생할수록, 불만 제기 확률이 높을 것이라는 가설을 세우고 이른 검정하기 위한 errtype 해석

### 5-1. errtype
1. errtype 종류에 상관없이 errtype의 수를 count

In [64]:
err_total = train_err.copy()

id_error = err_total[['user_id','errtype']].values
error = np.zeros(15000)

In [65]:
for person_idx, err,  in tqdm(id_error):
    error[person_idx-10000] += 1

100%|██████████| 16554663/16554663 [00:38<00:00, 431335.68it/s]


In [66]:
problem_user = sorted(list(train_prob.user_id.unique()))
prob_user = [i-10000 for i in problem_user]

In [67]:
psum = [] # 불만 제기한 사람들 각각의 Total Errtype Count
nsum = [] # 불만 제기하지않은 사람들 각각의 Total Errtype Count

for i in range(len(error)):
    if i in prob_user:
        psum.append(error[i])
    else:
        nsum.append(error[i])
        
print("불만 제기한 사람들의 Total Errtype Count의 평균은 {0}입니다.".format(sum(psum)/len(psum)))
print("불만 제기하지않은 사람들의 Total Errtype Count의 평균은 {0}입니다.".format(sum(nsum)/len(nsum)))

불만 제기한 사람들의 Total Errtype Count의 평균은 1517.7562입니다.
불만 제기하지않은 사람들의 Total Errtype Count의 평균은 896.5882입니다.


불만제기를 한 사람은 대부분 total errtype을 많이 가지고 있었음  
-> 따라서 errtype을 count하는 것이 유의미하다는 결론을 내리고, errtype의 평균, 최대값, 표준편차를 살펴보려함

2. errtype이 불만 제기에 중요한 변수임을 알았다. 그렇다면 errtype의 종류마다 영향력이 다를까?

In [68]:
type5_id = sorted(list(train_err[train_err['errtype'] == 17].user_id.unique()))
print("Errtype 5을 가진 유저들의 수 : {}명".format(len(type5_id)))
print("Errtype 5을 가진 유저중에서 불만을 제기한 사람의 수 : {}명".format(len(train_prob[train_prob['user_id'].isin(type5_id)].user_id.unique())))
print("비율 : %.3f%%"%(len(train_prob[train_prob['user_id'].isin(type5_id)].user_id.unique())/len(type5_id) * 100))

Errtype 5을 가진 유저들의 수 : 5747명
Errtype 5을 가진 유저중에서 불만을 제기한 사람의 수 : 2498명
비율 : 43.466%


In [69]:
type18_id = sorted(list(train_err[train_err['errtype'] == 18].user_id.unique()))
print("Errtype 18을 가진 유저들의 수 : {}명".format(len(type18_id)))
print("Errtype 18을 가진 유저중에서 불만을 제기한 사람의 수 : {}명".format(len(train_prob[train_prob['user_id'].isin(type18_id)].user_id.unique())))
print("비율 : %.3f%%"%(len(train_prob[train_prob['user_id'].isin(type18_id)].user_id.unique())/len(type18_id) * 100))

Errtype 18을 가진 유저들의 수 : 1768명
Errtype 18을 가진 유저중에서 불만을 제기한 사람의 수 : 1521명
비율 : 86.029%


-> 특정 errtype이 불만 제기에 큰 영향을 미침  
-> 따라서, errtype별 count를 통한 파생변수를 살펴볼 필요가 있음

3. errtype과 fwver&model_nm의 관계

In [70]:
# 모델 버전마다 Errtype이 다름을 보여주는 코드
for i in range(9):
    nm = 'model_' + str(i)
    print("모델 버전", nm, "→", sorted(list(train_err[train_err['model_nm'] == nm]['errtype'].unique()))[:15])

모델 버전 model_0 → [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
모델 버전 model_1 → [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
모델 버전 model_2 → [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
모델 버전 model_3 → [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
모델 버전 model_4 → [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
모델 버전 model_5 → [2, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
모델 버전 model_6 → [5]
모델 버전 model_7 → [3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18]
모델 버전 model_8 → [2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [71]:
# 펌웨어 버전마다 Errtype이 다름을 보여주는 코드
so_fw = sorted(list(train_err.fwver.unique()))

for i in so_fw:
    nm = i
    print("펌웨어 버전 →", i,sorted(list(train_err[train_err['fwver'] == nm]['errtype'].unique()))[:20])

펌웨어 버전 → 03.11.1141 [1, 5, 6, 7, 14, 27, 28, 30]
펌웨어 버전 → 03.11.1149 [1, 4, 5, 6, 7, 12, 14, 27, 28]
펌웨어 버전 → 03.11.1167 [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 27, 28, 30]
펌웨어 버전 → 04.16.2641 [3, 4, 6, 7, 11, 12, 13, 14, 15, 16, 22, 23, 26]
펌웨어 버전 → 04.16.3345 [3, 4, 10, 11, 12, 14, 15, 16, 26]
펌웨어 버전 → 04.16.3439 [6, 7, 11, 12, 14, 15, 16, 22, 23, 26, 27, 28, 31, 33, 34, 35, 38, 40]
펌웨어 버전 → 04.16.3553 [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
펌웨어 버전 → 04.16.3569 [5, 10, 11, 12, 15, 16, 17, 22, 23, 26, 31, 32, 33, 34, 42]
펌웨어 버전 → 04.16.3571 [2, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
펌웨어 버전 → 04.22.1442 [4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 18, 20, 22, 23, 25, 26]
펌웨어 버전 → 04.22.1656 [4, 5, 6, 7, 11, 12, 13, 14, 15, 16, 22, 23, 26, 27, 28]
펌웨어 버전 → 04.22.1666 [12, 13, 14, 38]
펌웨어 버전 → 04.22.1684 [1, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 22, 23, 25, 26, 27]
펌웨어 버전 → 04.22.1750 [1, 2, 4, 

### 5-2. ErrorCode

1. errtype과 비슷하게 errcode도 error의 정보를 알려주는 feature임. 먼저 둘의 관계를 살펴보려함



In [72]:
print(sorted(list(train_err.errcode.unique())[:50], reverse = True))
print("\nErrcode의 종류 : {}개".format(len(list(train_err.errcode.unique()))))

['terminate by peer user', 'standby', 'connectionterminated by local host', 'connection timeout', 'connection fail to establish', 'connection fail for LMP response timout', 'active', 'V-21008', 'UNKNOWN', 'U-81009', 'S-65002', 'S-64002', 'S-61001', 'Q-64002', 'NFANDROID2', 'J-30021', 'B-A8002', '95', '94', '93', '91', '90', '89', '88', '87', '86', '85', '84', '83', '82', '81', '80', '8.0', '79', '78', '77', '700001', '6796', '6467', '6', '5738', '5507', '4893', '4', '3', '2', '14', '13', '1', '0']

Errcode의 종류 : 2806개


In [73]:
ey_ec = train_err.groupby('errtype')['errcode'].unique().to_frame()
display(ey_ec)

,errcode
errtype,
1,"[0, P-44010, P-41011, P-41007 , P-44010 , P-41..."
2,"[1, 0]"
3,"[1, 2, 0]"
4,"[0, 1]"
5,"[B-A8002, Q-64002, S-61001, U-81009, V-21008, ..."
6,"[1, 14]"
7,"[1, 14]"
8,"[PHONE_ERR, PUBLIC_ERR, 20]"
9,"[V-21002, V-21005, 1, C-14014, V-21008, C-1203..."


-> errtype은 errcode를 군집화시킨 변수임  
  
2. errtype은 중요한 변수였고 errcode가 errtype에 속함.  
그렇다면 errcode도 종류별로 불만제기에 끼치는 영향이 다를까?

- 먼저, Errcode의 대표적인 종류는 다음과 같다.
    - P-41001과 같은 문자+숫자형 에러코드
    - 1, 13, 14와 같은 두자리수 이하의 에러코드
    - 3113, 3395와 같은 세자리수 이상의 에러코드
    - Connection Timeout, L2CAP Connection Cancelled와 같은 문장형 에러코드

In [74]:
ey_ec = ey_ec.reset_index()
display(sorted(list(ey_ec[ey_ec['errtype'] == 1].errcode.values[0][2:8])))
display(sorted(list(ey_ec[ey_ec['errtype'] == 5].errcode.values[0][6:12])))
display(sorted(list(ey_ec[ey_ec['errtype'] == 14].errcode.values[0][:])))
display(sorted(list(ey_ec[ey_ec['errtype'] == 25].errcode.values[0][3:9])))
display(sorted(list(ey_ec[ey_ec['errtype'] == 32].errcode.values[0][3:9])))
display(sorted(list(ey_ec[ey_ec['errtype'] == 38].errcode.values[0][3:9])))

['P-41001', 'P-41007', 'P-41007 ', 'P-41011', 'P-41011 ', 'P-44010 ']

['C-11017', 'H-51042', 'J-30021', 'Q-64001', 'S-64002', 'S-65002']

['1', '13', '14']

['L2CAP connection cancelled',
 'UNKNOWN',
 'connection fail to establish',
 'connection timeout',
 'connectionterminated by local host',
 'terminate by peer user']

['77', '78', '84', '85', '86', '90']

['3113', '3395', '3674', '39391', '4893', '5507']

-> 이 중 불만제기율이 가장 높은 errcode는??

In [75]:
problem = train_prob.copy()
p_id = sorted(problem['user_id'].unique())

errin = err_total[err_total['user_id'].isin(p_id)]['errcode'].value_counts().to_dict()
errout = err_total[~err_total['user_id'].isin(p_id)]['errcode'].value_counts().to_dict()

In [76]:
errcode_prob = []
most_error = []

for i in errin:
    try :
        errcode_prob.append((i,errin[i]/(errin[i]+errout[i]),errin[i],errout[i]))
    except :
        errcode_prob.append((i,1,errin[i],0))
    
for i in errout:
    if i in errin:
        continue
    errcode_prob.append((i,0,0,errout[i]))

In [77]:
errcode_prob.sort(key = lambda x: x[1], reverse = True)

In [78]:
most_error = []
most_not_error = []

for i in errcode_prob:
    if i[2] >= 20 and i[3] >= 20:
        if i[1] >= 0.75:
            most_error.append(i[0])
        elif i[1] <= 0.25 :
            most_not_error.append(i[0])

print("불만 제기 확률이 높은 에러코드 : {0}".format(most_error))
print("불만 제기 확률이 낮은 에러코드 : {0}".format(most_not_error))

불만 제기 확률이 높은 에러코드 : ['scanning timeout', '5', '6', 'V-21008', 'terminate by peer user', 'V-21005']
불만 제기 확률이 낮은 에러코드 : ['Q-64001', 'Q-64002', 'P-44010', 'PHONE_ERR', 'B-51049', 'H-51049']


**특정 errcode가 불만 제기에 유의미한 영향을 미침**을 알 수 있음  
-> 따라서 errcode와 불만제기 간의 관계를 알 수 있는 파생변수를 만들어야함

# 5-3. fwver
- 먼저, Fwver의 대표적인 종류는 다음과 같다.
    - '03'으로 시작하는 펌웨어 버전
    - '04'으로 시작하는 펌웨어 버전
    - '05'으로 시작하는 펌웨어 버전
    - '10', '8.5.3' 특수한 펌웨어 버전

In [79]:
print(sorted(train_err.fwver.unique()))

['03.11.1141', '03.11.1149', '03.11.1167', '04.16.2641', '04.16.3345', '04.16.3439', '04.16.3553', '04.16.3569', '04.16.3571', '04.22.1442', '04.22.1656', '04.22.1666', '04.22.1684', '04.22.1750', '04.22.1778', '04.33.1095', '04.33.1125', '04.33.1149', '04.33.1171', '04.33.1185', '04.33.1261', '04.73.2237', '04.73.2571', '04.82.1684', '04.82.1730', '04.82.1778', '05.15.2090', '05.15.2092', '05.15.2114', '05.15.2120', '05.15.2122', '05.15.2138', '05.15.3104', '05.66.3237', '05.66.3571', '10', '8.5.3']


1. 펌웨어 버전에 따라 불만제기율이 달라질까?

In [80]:
def relation_fw_complain(data): # 펌웨어 버젼과 불만 제기와의 관계
    fwlst = list(data['fwver'].value_counts().keys())
    result = []
    for i in tqdm(fwlst):
        f1 = sorted(list(data[data['fwver'] == i].user_id.unique()))
        f2 = sorted(list(train_prob[train_prob['user_id'].isin(f1)]['user_id'].unique()))
        if (len(f2)/len(f1) >= 0.5) and (len(f1) >= 5):
            result.append([i, len(f2)/len(f1), f2, f1])
    return result

In [81]:
def view_relation_fw_complain(data):
    result = relation_fw_complain(data)
    for i in range(len(result)):
        fw = result[i][0]
        fp = result[i][1] * 100
        fww = result[i][2]
        fwp = result[i][3]
        print("펌웨어 버전 %s를 사용중인 사람들 가운데, %.2f%%는 불만을 제기하였습니다. \n불만 제기를 한 사람의 수는 전체 %s명중 %s명입니다.\n"%(fw, fp, len(fwp), len(fww)))

In [82]:
view_relation_fw_complain(train_err)

100%|██████████| 37/37 [00:39<00:00,  1.08s/it]

펌웨어 버전 04.16.3571를 사용중인 사람들 가운데, 58.76%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 502명중 295명입니다.

펌웨어 버전 05.66.3237를 사용중인 사람들 가운데, 52.46%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 61명중 32명입니다.

펌웨어 버전 04.82.1778를 사용중인 사람들 가운데, 50.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 40명중 20명입니다.

펌웨어 버전 05.66.3571를 사용중인 사람들 가운데, 53.33%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 15명중 8명입니다.

펌웨어 버전 04.33.1149를 사용중인 사람들 가운데, 70.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 160명중 112명입니다.

펌웨어 버전 04.73.2571를 사용중인 사람들 가운데, 66.67%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 12명중 8명입니다.

펌웨어 버전 04.33.1125를 사용중인 사람들 가운데, 66.67%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 6명중 4명입니다.



**특정 펌웨어 버전이 불만제기에 영향을 미침**을 알 수 있다.
-> 따라서 펌웨어 버전과 불만제기율 간의 관계를 알 수 있는 파생변수를 만들어야 함

In [83]:
view_relation_fw_complain(train_qual)

100%|██████████| 27/27 [00:02<00:00, 10.24it/s]

펌웨어 버전 04.22.1684를 사용중인 사람들 가운데, 59.09%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 22명중 13명입니다.

펌웨어 버전 09.17.1431를 사용중인 사람들 가운데, 96.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 25명중 24명입니다.

펌웨어 버전 04.16.3571를 사용중인 사람들 가운데, 63.24%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 68명중 43명입니다.

펌웨어 버전 04.82.1684를 사용중인 사람들 가운데, 54.55%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 22명중 12명입니다.

펌웨어 버전 04.33.1149를 사용중인 사람들 가운데, 70.27%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 37명중 26명입니다.

펌웨어 버전 05.66.3237를 사용중인 사람들 가운데, 75.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 8명중 6명입니다.

펌웨어 버전 04.82.1778를 사용중인 사람들 가운데, 61.54%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 13명중 8명입니다.

펌웨어 버전 05.66.3571를 사용중인 사람들 가운데, 60.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 5명중 3명입니다.



2. fwver 앞 4자리만 같고, 뒤 4자리가 다른 버전들이 존재 -> 펌웨어 버전의 업데이트가 영향을 미칠까?

In [84]:
train_fw = train_err.copy()

In [85]:
train_fw['fwver'] = train_fw['fwver'].apply(lambda x : x.replace(".",""))
train_fw['fwver'] = train_fw['fwver'].apply(lambda x : int(x))

In [86]:
te_uf = train_fw[['user_id', 'fwver']]
fw_df = ~(te_uf == te_uf.shift(1))

In [88]:
logical = (fw_df.user_id.apply(int) + fw_df.fwver.apply(int)) > 0
tre_fw_counts = te_uf[logical]

In [90]:
user_id_2_fw = tre_fw_counts.user_id.value_counts()
user_id_lst_2_fw = list(user_id_2_fw.loc[user_id_2_fw == 2].to_frame().index)

In [91]:
sort_two_fw_user = train_fw.loc[train_fw.user_id.isin(user_id_lst_2_fw)].drop_duplicates(['user_id','fwver'],keep='last').drop_duplicates('user_id',keep='first').sort_values("time").user_id

In [92]:
operator1 = len(user_id_lst_2_fw)
operator2 = len(list(train_prob[train_prob['user_id'].isin(user_id_lst_2_fw)].user_id.unique()))

print("펌웨어 버전이 변하는 시기에 불만을 제기한 사람 {0}%".format(round(operator2 / operator1 * 100,2)))

펌웨어 버전이 변하는 시기에 불만을 제기한 사람 36.45%


**펌웨어 버전 업데이트가 불만 제기에 유의미한 영향을 미침**을 알 수 있다.  
-> 따라서 펌웨어 버전 변화에 따른 불만 제기를 알 수 있는 파생변수를 만들어야 함을 알 수 있다.

# 5-4 Model(모델 버전)
- 먼저, Model의 종류는 다음과 같다ㅏ.
    - Model_0 부터 Model_8까지 총 9가지 종류의 모델 넘버가 존재

In [93]:
print(sorted(list(train_err.model_nm.unique())))

['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_6', 'model_7', 'model_8']


1. 모델 버전과 펌웨어 버전의 관계

In [94]:
display(train_err.groupby('model_nm')['fwver'].unique().to_frame().reset_index())

,model_nm,fwver
0,model_0,"[04.22.1750, 04.22.1778, 04.22.1684, 04.22.166..."
1,model_1,"[04.16.3553, 04.16.3571, 04.16.3439, 04.16.356..."
2,model_2,"[04.33.1185, 04.33.1261, 04.33.1149, 04.33.117..."
3,model_3,"[05.15.2138, 05.15.2120, 05.15.2090, 05.15.310..."
4,model_4,"[03.11.1149, 03.11.1167, 03.11.1141]"
5,model_5,"[04.82.1684, 04.82.1778, 04.82.1730]"
6,model_6,"[10, 8.5.3]"
7,model_7,"[05.66.3237, 05.66.3571]"
8,model_8,"[04.73.2237, 04.73.2571]"


In [96]:
# 모델마다 펌웨어 버전이 다름을 보여주는 코드
for i in range(9):
    nm = 'model_' + str(i)
    print("모델 :",nm, "->\n", sorted(list(train_err[train_err['model_nm']==nm]['fwver'].unique())))

모델 : model_0 ->
 ['04.22.1442', '04.22.1656', '04.22.1666', '04.22.1684', '04.22.1750', '04.22.1778']
모델 : model_1 ->
 ['04.16.2641', '04.16.3345', '04.16.3439', '04.16.3553', '04.16.3569', '04.16.3571']
모델 : model_2 ->
 ['04.33.1095', '04.33.1125', '04.33.1149', '04.33.1171', '04.33.1185', '04.33.1261']
모델 : model_3 ->
 ['05.15.2090', '05.15.2092', '05.15.2114', '05.15.2120', '05.15.2122', '05.15.2138', '05.15.3104']
모델 : model_4 ->
 ['03.11.1141', '03.11.1149', '03.11.1167']
모델 : model_5 ->
 ['04.82.1684', '04.82.1730', '04.82.1778']
모델 : model_6 ->
 ['10', '8.5.3']
모델 : model_7 ->
 ['05.66.3237', '05.66.3571']
모델 : model_8 ->
 ['04.73.2237', '04.73.2571']


-> 모델과 펌웨어 버전은 종속적인 관계로, 모델별 펌웨어 버전이 존제함을 알 수 있다.

2. 모델 버전 변경의 영향력

In [97]:
train_md = train_err.copy()
train_md['model_nm'] = train_md['model_nm'].apply(lambda x : x.replace("_",""))
train_md['model_nm'] = train_md['model_nm'].apply(lambda x : str(x)[5])

In [98]:
te_uf = train_md[['user_id', 'model_nm']]
md_df = ~(te_uf == te_uf.shift(1))

In [99]:
logical = (md_df.user_id.apply(int) + md_df.model_nm.apply(int)) > 0
tre_md_counts = te_uf[logical]

In [101]:
user_id_2_md = tre_md_counts.user_id.value_counts()
user_id_lst_2_md = list(user_id_2_md.loc[user_id_2_md == 2].to_frame().index)

In [103]:
sort_two_md_user = train_md.loc[train_md.user_id.isin(user_id_lst_2_md)].drop_duplicates(['user_id', 'model_nm'],keep='last').drop_duplicates('user_id',keep='first').sort_values('time').user_id

In [104]:
operator1_m = len(user_id_lst_2_md)
operator2_m = len(list(train_prob[train_prob['user_id'].isin(user_id_lst_2_md)].user_id.unique()))

print("모델이 변하는 시기에 불만을 제기한 사람 {0}%".format(round(operator2_m / operator1_m * 100,2)))

모델이 변하는 시기에 불만을 제기한 사람 91.57%


##### 모델 버전의 변경은 불만제기에 큰 영향을 미친다
-> 따라서 모델 버전의 변화와 불만제기 간의 관계를 알 수 있는 파생변수를 만들어야함

# 6. quality data 수치 분석

In [105]:
train_qual.iloc[:,[1,0,3,4,5,6,7,8,9,10,11,12,13,14,15]][12372:12390]

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
12372,10265,2020-11-08 07:50:00,0,0,0,0,0,0,0,13,1,119,132,0,2
12373,10265,2020-11-08 07:50:00,0,0,1,0,0,4,1,13,1,119,132,1,2
12374,10265,2020-11-08 07:50:00,0,0,4,0,0,12,0,13,1,119,132,1,2
12375,10265,2020-11-08 07:50:00,0,1,21,0,0,23,12,13,1,119,132,0,2
12376,10265,2020-11-08 07:50:00,0,0,54,0,0,54,0,13,1,119,132,0,2
12377,10265,2020-11-08 07:50:00,0,0,0,0,0,0,0,13,1,119,132,0,2
12378,10265,2020-11-08 07:50:00,0,0,37,0,0,37,0,13,1,119,132,0,2
12379,10265,2020-11-08 07:50:00,0,0,0,0,0,0,0,13,1,119,132,0,2
12380,10265,2020-11-08 07:50:00,0,0,0,0,0,0,0,13,1,119,132,0,2
12381,10265,2020-11-08 07:50:00,0,0,0,0,0,0,0,13,1,119,132,0,2


In [106]:
train_qual.iloc[:,[1,0,3,4,5,6,7,8,9,10,11,12,13,14,15]].describe()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000
mean,17583.75951,3.59968,-0.16982,4.75090,0.00000,0.00000,78.14516,2.11059,27.52441,0.15948,59.20809,939.93923,-0.17910,0.04810
std,4400.24714,446.47710,0.69788,586.24020,0.00000,0.00000,2335.76090,33.45625,325.64862,5.27364,3362.65838,16934.49764,0.39630,0.30941
min,10000.00000,-1.00000,-1.00000,-1.00000,0.00000,0.00000,-1.00000,-1.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
25%,13695.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,3.00000,0.00000,0.00000
50%,17507.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,6.00000,0.00000,0.00000
75%,21532.50000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,41.00000,0.00000,0.00000
max,24997.00000,157667.00000,171.00000,191859.00000,0.00000,0.00000,637385.00000,600.00000,7200.00000,1317.00000,397424.00000,1910175.00000,14.00000,14.00000


# 6-1. quality 값
1. quality는 측정가능한 지표로 만들어진 값이다. 그러나 해당 값들의 범위를 살펴보면 [0,무한대)가 아닌 [-1,무한대)이다.
"-1 값은 측정이 되지 못한, 알 수 없음(또는 문제 있음)을 나타내는 지표일 것이다" 라는 가설을 세움
-> -1값을 많이 갖고 있는 유저가 불만제기를 더 많이 하였는지 확인

In [107]:
# -1을 갖고 있는 유저의 수
len(sorted(list(train_qual[train_qual['quality_0'] == -1].groupby('user_id')['quality_0'].count().to_frame().reset_index().sort_values(ascending=False, by='quality_0').user_id.unique())))

5567

In [111]:
# -1을 많이 가지고 있는 유저 상위 100명
minus_id_top100 = list(train_qual[train_qual['quality_0'] == -1].groupby('user_id')['quality_0'].count().to_frame().reset_index().sort_values(ascending=False, by = 'quality_0').user_id.unique())[:100]

In [112]:
# 상위 100명중, 불만을 제기한 사람의 수 : 70명 (70.0%)
len(train_prob[train_prob['user_id'].isin(minus_id_top100)].user_id.unique())

70

In [113]:
# -1을 많이 가지고 있는 유저 상위 900명
minus_id_top900 = list(train_qual[train_qual['quality_0'] == -1].groupby('user_id')['quality_0'].count().to_frame(). \
reset_index().sort_values(ascending=False, by='quality_0').user_id.unique())[100:1000]



In [114]:
# 상위 900명중, 불만을 제기한 사람의 수 : 505명 (56.1%)
len(train_prob[train_prob['user_id'].isin(minus_id_top900)].user_id.unique())

505

In [115]:
# -1을 많이 가지고 있는 유저 상위 2000명
minus_id_top2000 = list(train_qual[train_qual['quality_0'] == -1].groupby('user_id')['quality_0'].count().to_frame(). \
reset_index().sort_values(ascending=False, by='quality_0').user_id.unique())[1000:3000]

In [116]:
# 상위 2000명중, 불만을 제기한 사람의 수 : 799명 (39.9%)
len(train_prob[train_prob['user_id'].isin(minus_id_top2000)].user_id.unique())

799

##### "-1을 더 많이 가지고 있을수록 불만을 더 많이 제기함"
-> 따라서 -1은 불만 제기와 관련이 있음 -> 파생변수를 만들어야함

2. -1 값을 카운팅 한 뒤, 해당 값들을 제외한 나머지 변수들의 수치를 해석하기 위하여 -1을 0으로 변경

In [118]:
for i in train_qual.columns[3:]:
    idx = train_qual[train_qual[i] == -1].index
    train_qual.loc[idx, i] = 0

3. quality data 기본 수치 해석

In [119]:
train_qual.iloc[:,3:].describe()

,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000
mean,3.76559,0.01329,4.93401,0.00000,0.00000,78.32827,2.29370,27.52441,0.15948,59.20809,939.93923,0.00401,0.04810
std,446.47555,0.57670,586.23853,0.00000,0.00000,2335.75473,33.44146,325.64862,5.27364,3362.65838,16934.49764,0.07747,0.30941
min,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,3.00000,0.00000,0.00000
50%,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,6.00000,0.00000,0.00000
75%,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,41.00000,0.00000,0.00000
max,157667.00000,171.00000,191859.00000,0.00000,0.00000,637385.00000,600.00000,7200.00000,1317.00000,397424.00000,1910175.00000,14.00000,14.00000


→   Quality_3과 Quality_4는 모두 0값만을 가지고 있음을 확인  
→   Quality_0, Quality_2, Quality_5, Quality_9, Quality_10의 분산과 최댓값이 크다는 것을 확인 : 불만 제기와의 관계 확인

In [120]:
# Quality_2, 5, 9, 10과 불만 제기와의 관계 확인
for i in [0, 2, 5, 9, 10]:
    tq = train_qual.groupby('user_id')['quality_'+str(i)].max().to_frame().reset_index().sort_values(ascending=False, by='quality_'+str(i))
    tq_id = list(tq.user_id.values)[100:900]
    tp_id = train_prob[train_prob['user_id'].isin(tq_id)].user_id.unique()
    print("퀄리티 {0}을 합한 값이 큰 상위 유저 900명 중 {1}명이 불만을 제기하였습니다.  {2}%".format(i, len(tp_id), len(tp_id)/len(tq_id)*100))

퀄리티 0을 합한 값이 큰 상위 유저 900명 중 290명이 불만을 제기하였습니다.  36.25%
퀄리티 2을 합한 값이 큰 상위 유저 900명 중 284명이 불만을 제기하였습니다.  35.5%
퀄리티 5을 합한 값이 큰 상위 유저 900명 중 375명이 불만을 제기하였습니다.  46.875%
퀄리티 9을 합한 값이 큰 상위 유저 900명 중 282명이 불만을 제기하였습니다.  35.25%
퀄리티 10을 합한 값이 큰 상위 유저 900명 중 401명이 불만을 제기하였습니다.  50.125%


In [121]:
# Quality_2, 5, 9, 10과 불만 제기와의 관계 확인
for i in [0, 2, 5, 9, 10]:
    tq = train_qual.groupby('user_id')['quality_'+str(i)].max().to_frame().reset_index().sort_values(ascending=False, by='quality_'+str(i))
    tq_id = list(tq.user_id.values)[:100]
    tp_id = train_prob[train_prob['user_id'].isin(tq_id)].user_id.unique()
    print("퀄리티 {0}을 합한 값이 큰 상위 유저 100명 중 {1}명이 불만을 제기하였습니다.  {2}%".format(i, len(tp_id), len(tp_id)/len(tq_id)*100))

퀄리티 0을 합한 값이 큰 상위 유저 100명 중 36명이 불만을 제기하였습니다.  36.0%
퀄리티 2을 합한 값이 큰 상위 유저 100명 중 39명이 불만을 제기하였습니다.  39.0%
퀄리티 5을 합한 값이 큰 상위 유저 100명 중 74명이 불만을 제기하였습니다.  74.0%
퀄리티 9을 합한 값이 큰 상위 유저 100명 중 39명이 불만을 제기하였습니다.  39.0%
퀄리티 10을 합한 값이 큰 상위 유저 100명 중 74명이 불만을 제기하였습니다.  74.0%


→   퀄리티 합이 높을수록, 불만 제기 확률이 높아짐

4. 나머지 Quality 변수들은 0부터 무한대 사이의 값을 가지고 있으므로, 이것이 시스템 품질을 측정한 하나의 값으로 가정  
→   Quality 변수들의 간의 관계를 해석하기 위하여 Correlation을 이용  
→   유저별 Quality값들의 합이 높은 것과 불만 제기의 관계 해석

In [122]:
qual_sum = pd.DataFrame({'user_id' : np.arange(10000,25000)})

for i in train_qual.columns[3:]:
    qq = train_qual.groupby('user_id')[i].sum().to_frame().reset_index()
    qual_sum = pd.merge(qual_sum, qq, how='left', on ='user_id')
    qual_sum = qual_sum.fillna(0)

In [123]:
qual_sum.head()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,0.00000,0.00000,0.00000,0.00000,0.00000,12.00000,0.00000,0.00000,0.00000,0.00000,144.00000,0.00000,0.00000
1,10001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,10002,2.00000,0.00000,1.00000,0.00000,0.00000,31.00000,46.00000,552.00000,0.00000,12.00000,372.00000,0.00000,0.00000
3,10003,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,10004,0.00000,0.00000,0.00000,0.00000,0.00000,4.00000,87.00000,1044.00000,0.00000,0.00000,48.00000,0.00000,0.00000


In [124]:
qual_sum.corr()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
user_id,1.00000,0.01775,0.00618,0.01177,NaN,NaN,-0.00422,-0.00978,-0.00978,0.00618,0.01177,-0.00422,0.00959,0.00959
quality_0,0.01775,1.00000,-0.00106,0.88105,NaN,NaN,0.04295,0.00066,0.00066,-0.00106,0.88105,0.04295,0.00106,0.00106
quality_1,0.00618,-0.00106,1.00000,-0.00035,NaN,NaN,0.02997,0.00733,0.00733,1.00000,-0.00035,0.02997,0.10275,0.10275
quality_2,0.01177,0.88105,-0.00035,1.00000,NaN,NaN,0.04433,0.00107,0.00107,-0.00035,1.00000,0.04433,0.02326,0.02326
quality_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
quality_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
quality_5,-0.00422,0.04295,0.02997,0.04433,NaN,NaN,1.00000,0.01970,0.01970,0.02997,0.04433,1.00000,0.07244,0.07244
quality_6,-0.00978,0.00066,0.00733,0.00107,NaN,NaN,0.01970,1.00000,1.00000,0.00733,0.00107,0.01970,0.00061,0.00061
quality_7,-0.00978,0.00066,0.00733,0.00107,NaN,NaN,0.01970,1.00000,1.00000,0.00733,0.00107,0.01970,0.00061,0.00061
quality_8,0.00618,-0.00106,1.00000,-0.00035,NaN,NaN,0.02997,0.00733,0.00733,1.00000,-0.00035,0.02997,0.10275,0.10275


→ Quality 값을 유저별로 합하여 만든 파생 변수들 중에, 완벽한 선형관계를 보이는 관계를 발견

In [125]:
display(qual_sum.iloc[:,[2,9]][2505:2510])
display(qual_sum.iloc[:,[3,10]][250:255])
display(qual_sum.iloc[:,[6,11]][2505:2510])
display(qual_sum.iloc[:,[7,8]][2505:2510])
display(qual_sum.iloc[:,[12,13]][150:155])

,quality_1,quality_8
2505,4.00000,48.00000
2506,0.00000,0.00000
2507,0.00000,0.00000
2508,2.00000,24.00000
2509,0.00000,0.00000


,quality_2,quality_9
250,0.00000,0.00000
251,70.00000,840.00000
252,0.00000,0.00000
253,0.00000,0.00000
254,0.00000,0.00000


,quality_5,quality_10
2505,10.00000,120.00000
2506,5.00000,60.00000
2507,0.00000,0.00000
2508,0.00000,0.00000
2509,0.00000,0.00000


,quality_6,quality_7
2505,4.00000,48.00000
2506,1.00000,12.00000
2507,0.00000,0.00000
2508,0.00000,0.00000
2509,0.00000,0.00000


,quality_11,quality_12
150,1.00000,12.00000
151,0.00000,0.00000
152,0.00000,0.00000
153,0.00000,0.00000
154,0.00000,0.00000
